![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_Hubert.ipynb)

# Import ONNX Hubert models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- The Hubert model was introduced in `Spark NLP 5.1.0 and requires Spark version 3.4.1 and up.`
- Official models are supported, but not all custom models may work.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to export ONNX models directly via the CLI with `!optimum-cli export onnx --model MODEL_NAME EXPORT_PATH`.
- We'll use the [facebook/hubert-large-ls960-ft](https://huggingface.co/facebook/hubert-large-ls960-ft) model from HuggingFace as an example and export it with the `optimum-cli`.

In [2]:
MODEL_NAME = "facebook/hubert-large-ls960-ft"
EXPORT_PATH = f"export_onnx/{MODEL_NAME}"

! optimum-cli export onnx --model {MODEL_NAME} {EXPORT_PATH}

2025-06-14 03:17:17.151499: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749871037.485760    1241 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749871037.577963    1241 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-14 03:17:18.295364: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
config.json: 100% 1.38k/1.38k [00:00<00:00, 6.27MB/s]
pytorch_model.bin: 100% 1.26G/1.26G [00:58<00:00, 21.6MB/s]
model.safet

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 1232836
-rw-r--r-- 1 root root       1600 Jun 14 03:18 config.json
-rw-r--r-- 1 root root 1262397420 Jun 14 03:18 model.onnx
-rw-r--r-- 1 root root        212 Jun 14 03:18 preprocessor_config.json
-rw-r--r-- 1 root root         96 Jun 14 03:18 special_tokens_map.json
-rw-r--r-- 1 root root       1120 Jun 14 03:18 tokenizer_config.json
-rw-r--r-- 1 root root        358 Jun 14 03:18 vocab.json


We have to move additional model assets into a seperate folder, so that Spark NLP can load it properly.

In [4]:
! mkdir -p {EXPORT_PATH}/assets

In [5]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
processor.save_pretrained(f"{EXPORT_PATH}/assets")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[]

In [6]:
!ls -l {EXPORT_PATH}/assets

total 16
-rw-r--r-- 1 root root  254 Jun 14 03:20 preprocessor_config.json
-rw-r--r-- 1 root root   96 Jun 14 03:20 special_tokens_map.json
-rw-r--r-- 1 root root 1120 Jun 14 03:20 tokenizer_config.json
-rw-r--r-- 1 root root  358 Jun 14 03:20 vocab.json


## Import and Save Hubert in Spark NLP

- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [7]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 32.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `HubertForCTC` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `HubertForCTC` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [9]:
from sparknlp.annotator import HubertForCTC

hubert = (
    HubertForCTC.loadSavedModel(f"{EXPORT_PATH}", spark)
    .setInputCols("audio_assembler")
    .setOutputCol("text")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
hubert.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX Hubert model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {MODEL_NAME}_spark_nlp

total 2466016
drwxr-xr-x 3 root root       4096 Jun 14 03:24 fields
-rw-r--r-- 1 root root 1262590184 Jun 14 03:25 hubert_ctc_onnx
drwxr-xr-x 2 root root       4096 Jun 14 03:24 metadata
-rw-r--r-- 1 root root 1262590184 Jun 14 03:24 wav_ctc_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny Hubert model 😊

In [13]:
! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt

--2025-06-14 03:25:49--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2199992 (2.1M) [text/plain]
Saving to: ‘librispeech_asr_0.txt’

librispeech_asr_0.t 100%[===================>]   2.10M  --.-KB/s    in 0.07s   

2025-06-14 03:25:49 (29.9 MB/s) - ‘librispeech_asr_0.txt’ saved [2199992/2199992]



In [14]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

audioAssembler = AudioAssembler()\
    .setInputCol("audio_content")\
    .setOutputCol("audio_assembler")

speechToText = HubertForCTC.load(f"{MODEL_NAME}_spark_nlp")

pipeline = Pipeline().setStages([
    audioAssembler,
    speechToText
])

with open("librispeech_asr_0.txt") as f:
    raw_floats = [float(x) for x in f.read().strip().split("\n")]

processedAudioFloats = spark.createDataFrame([[raw_floats]]).toDF("audio_content")
result = pipeline.fit(processedAudioFloats).transform(processedAudioFloats)

result.select("text.result").show(truncate=False)

+------------------------------------------------------------------------------------------+
|result                                                                                    |
+------------------------------------------------------------------------------------------+
|[MISTER QUILTER IS THE APOSTLE OF THE MIDLE CLASES AND WE ARE GLAD TO WELCOME HIS GOSPEL ]|
+------------------------------------------------------------------------------------------+



That's it! You can now go wild and use hundreds of Hubert models from HuggingFace 🤗 in Spark NLP 🚀
